<a href="https://colab.research.google.com/github/shashankbhagat/Recommendation-System/blob/master/Recommendation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import csv
from collections import defaultdict
import random
import scipy as sc
import tensorflow as tf
import tensorrec
#!pip install tensorrec

In [4]:
print('Loading Data')
with open('ratings.csv') as ratings_file:
    ratings_file_reader=csv.reader(ratings_file)
    raw_ratings = list(ratings_file_reader)
    raw_ratings_header = raw_ratings.pop(0)

Loading Data


In [0]:
# Iterate through the input to map MovieLens IDs to new internal IDs
# The new internal IDs will be created by the defaultdict on insertion

movielens_to_internal_user_ids=defaultdict(lambda:len(movielens_to_internal_user_ids))
movielens_to_internal_item_ids=defaultdict(lambda:len(movielens_to_internal_item_ids))
for row in raw_ratings:
    row[0] = movielens_to_internal_user_ids[int(row[0])]
    row[1] = movielens_to_internal_item_ids[int(row[1])]
    row[2]=float(row[2])
n_users = len(movielens_to_internal_user_ids)
n_items=len(movielens_to_internal_item_ids)

In [0]:
random.shuffle(raw_ratings)
cutoff=int(0.8*len(raw_ratings))
train_ratings=raw_ratings[:cutoff]
test_ratings=raw_ratings[cutoff:]

In [0]:
# This method converts a list of (user, item, rating, time) to a sparse matrix
def interactions_list_to_sparse_matrix(interactions):
    user_column,item_column, ratings_column, _ = zip(*interactions)
    return sc.sparse.coo_matrix((ratings_column,(user_column,item_column)),shape=(n_users,n_items))

# Create sparse matrices of interaction data
sparse_train_ratings = interactions_list_to_sparse_matrix(train_ratings)
sparse_test_ratings = interactions_list_to_sparse_matrix(test_ratings)

In [9]:
# Construct indicator features for users and items
user_indicator_features = sc.sparse.identity(n_users)
item_indicator_features = sc.sparse.identity(n_items)

# Build a matrix factorization collaborative filter model
cf_model = tensorrec.TensorRec(n_components=5)

#Fit the collaborative filter model
print('Training collaborative filter')
cf_model.fit(interactions=sparse_train_ratings,
             user_features=user_indicator_features, 
             item_features=item_indicator_features)

Training collaborative filter


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


In [15]:
# Create sets of train/test interactions that are only ratings >= 4.0
sparse_train_ratings_4plus=sparse_train_ratings.multiply(sparse_train_ratings>=4.0)
sparse_test_ratings_4plus=sparse_test_ratings.multiply(sparse_test_ratings>=4.0)

# This method consumes item ranks for each user and prints out recall@10 train/test metrics
def check_results(ranks):
    train_recall_at_10 = tensorrec.eval.recall_at_k(test_interactions=sparse_test_ratings_4plus,
                                                    predicted_ranks=ranks,
                                                   k=10).mean()
    test_recall_at_10 = tensorrec.eval.recall_at_k(test_interactions=sparse_train_ratings_4plus,
                                                    predicted_ranks=ranks,
                                                   k=10).mean()
    
    print("Recall at 10: Train: {:.4f} Test: {:.4f}".format(train_recall_at_10,test_recall_at_10))
    
#Check result of CF
print("Matrix fatorization collaborative filter:")
predicted_ranks = cf_model.predict_rank(user_features=user_indicator_features,
                                           item_features = item_indicator_features)
check_results(predicted_ranks)    

Matrix fatorization collaborative filter:
Recall at 10: Train: 0.0017 Test: 0.0008


In [20]:
#Another loss function: WMRB
print('Training collaborative filter with WMRB loss')
ranking_cf_model = tensorrec.TensorRec(n_components=5,loss_graph=tensorrec.loss_graphs.WMRBLossGraph())
ranking_cf_model.fit(interactions=sparse_train_ratings_4plus, user_features=user_indicator_features, item_features=item_indicator_features, n_sampled_items=int(n_items*.01))

#Check results of the WMRB Matrix Factorization CF model
predicted_ranks = ranking_cf_model.predict_rank(user_features=user_indicator_features, 
                                                item_features=item_indicator_features)
check_results(predicted_ranks)


Training collaborative filter with WMRB loss


/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "
/usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/gradients_util.py:93: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Recall at 10: Train: 0.0822 Test: 0.1223
